In [123]:
import math
import random
import numpy as np
import sys
import copy

In [419]:
file_path_tmp = "../../data/mtsp51.txt"
def load_data(file_path_tmp):
    row_data = []
    with open(file_path_tmp, "r") as f:
        row_data = f.readlines()
    
    row_num = len(row_data) - 1
    
    iso_tuple_list = []
    for i in range(1, len(row_data)):
        item = row_data[i].strip().split(" ")
        iso_tuple_list.append((int(item[0]), int(item[1]), int(item[2])))
    return iso_tuple_list

def cal_dis(t1, t2):
    return math.sqrt(sum([(a - b)**2 for (a,b) in zip(t1[1:],t2[1:])]))

In [420]:
row_data = load_data(file_path_tmp)

N = len(row_data)
M = 5
POPULATION = 100
RUNS = 30

print(row_data[0], row_data[50])
print(cal_dis(row_data[0], row_data[1]))

(1, 37, 52) (51, 30, 40)
12.36931687685298


## Initial

In [439]:
def generate_breaks(n, m, min_nodes, max_nodes = None):
#     breakpoints = sorted(random.sample(range(n), m - 1))
    
    valid = False
    while valid is False:
        valid = True
#         sorted
        breakpoints = sorted(random.sample(range(n), m - 1))
    
#          first
        if breakpoints[0] < min_nodes or (max_nodes is not None and breakpoints[0] > max_nodes):
            valid = False
            continue
            
#         middle
        for i in range(0, len(breakpoints)-1):
            if breakpoints[i+1] - breakpoints[i] < min_nodes or (max_nodes is not None and breakpoints[i+1] - breakpoints[i] > max_nodes):
                valid = False
                break
        
        if not valid:
            continue
# last
        if n - breakpoints[-1] < min_nodes or (max_nodes is not None and n - breakpoints[-1] > max_nodes):
            valid = False
    return breakpoints

In [470]:
def generate_populations(population_size, n, m, min_nodes, max_nodes = None):
    populations = [random.sample(range(n), n) for i in range(population_size)] 
    breaks = [generate_breaks(n, m, min_nodes, max_nodes) for i in range(population_size)]
    return populations, breaks

def cal_fitness(population, break_points, n):
    paths = [0] + break_points + [n]
    fitness_tmp = 0
    for i in range(len(paths) - 1):
        begin = paths[i]#include
        end = paths[i + 1]#not include
        tmp_index = 0
        for j in range(begin, end):
            fitness_tmp += cal_dis(row_data[population[j]], row_data[tmp_index])
            tmp_index = population[j]
        fitness_tmp += cal_dis(row_data[0], row_data[tmp_index])
    return fitness_tmp

## Mutation Operators

In [460]:
def flip_insert(population, ij, p):
    population_tmp = copy.deepcopy(population[ij[0]:ij[1] + 1])
    population_tmp.reverse()
    
    del population[ij[0]:ij[1] + 1]
    
    population[p : p] = population_tmp
    
    
    
def swap_insert(population, ij, p):
    population_tmp = copy.deepcopy(population[ij[0]:ij[1] + 1])
    population_tmp[0],population_tmp[-1] = population_tmp[-1], population_tmp[0]
    
    del population[ij[0]:ij[1] + 1]
    
    population[p : p] = population_tmp
    
def left_lide_insert(population, ij, p):
    population_tmp = copy.deepcopy(population[ij[0]:ij[1] + 1])
    population_tmp.append(population_tmp.pop(0))
    
    del population[ij[0]:ij[1] + 1]
    
    population[p : p] = population_tmp
    

def right_lide_insert(population, ij, p):
    population_tmp = copy.deepcopy(population[ij[0]:ij[1] + 1])
    population_tmp.insert(0, population_tmp.pop(0))
    
    del population[ij[0]:ij[1] + 1]
    
    population[p : p] = population_tmp
    

In [550]:

def IPGA(populations,breaks, fitness_list, n, m, min_nodes, max_nodes):
#     print("-------------------IPGA----------------------")
#     print('in IPGA',min(fitness_list))
    best_index_global = 0
    for i in range(len(fitness_list)):
        if fitness_list[i] == min(fitness_list):
            best_index_global = i
            
#     fitness_list_tmp = [cal_fitness(populations[j], breaks[j], N) for j in range(POPULATION)]
#     print('in IPGA Cal',min(fitness_list_tmp))
    offspring = []
    breaks_offspring = []
    
#     print("********************shuffle****************")
    
    shuffle = random.sample(range(len(populations)), len(populations))
    total = []
    
    for i in range(0, len(populations), 10):
#         print("round: ",i)
        ij = sorted(random.sample(range(n), 2))
        p = random.randint(0, n)
    
        
        index_tmp = shuffle[i:i + 10]
        minus = 3000000
        index_best = -1 
        best_populations = []
        breaks_offspring_tmp = []
        
        for j in range(10):
            index = index_tmp[j]
#             if index == best_index_global:
#                 print(">>>>>>>>>>>>>>>>>>>>here", fitness_list[index], best_index_global, index_best)
                
            if fitness_list[index] <= minus:
                minus = fitness_list[index]
                index_best = index 
            
#             if index == best_index_global:
#                 print(">>>>>>>>>>>>>>>>>>>>here", fitness_list[index], best_index_global, index_best)
        
#         print(index_tmp)
#         print(fitness_list[index_best])
#         if index_best == best_index_global:
#             print("fuckkkkkkkkkk",fitness_list[index_best])
    
        best_populations = [copy.deepcopy(populations[index_best]) for j in range(10)]
        breaks_offspring_tmp = [copy.deepcopy(breaks[index_best]) for j in range(10)] 
        
#         1, nothing
        
#         2. FlipInsert
        flip_insert(best_populations[1], ij, p)

        
#         3. SwapInsert
        swap_insert(best_populations[2], ij, p)

#         4. LSlideInsert

        left_lide_insert(best_populations[3], ij, p)
        
#         5. RSlideInsert
        right_lide_insert(best_populations[4], ij, p)
        
#         6.modify breaks 
        breaks_offspring_tmp [5] = generate_breaks(n, m, min_nodes, max_nodes)
    
#         7. breaks + flip
        flip_insert(best_populations[6], ij, p)
        breaks_offspring_tmp [6] = generate_breaks(n, m, min_nodes, max_nodes)
        
#         8. breaks + swap
        swap_insert(best_populations[7], ij, p)
        breaks_offspring_tmp [7] = generate_breaks(n, m, min_nodes, max_nodes)
        
#         9. breaks + left slide
        left_lide_insert(best_populations[8], ij, p)
        breaks_offspring_tmp [8] = generate_breaks(n, m, min_nodes, max_nodes)

#         10. breaks + right slide
        right_lide_insert(best_populations[9], ij, p)
        breaks_offspring_tmp [9] = generate_breaks(n, m, min_nodes, max_nodes)
        
        breaks_offspring += [copy.deepcopy(breaks_offspring_tmp[t]) for t in range(len(breaks_offspring_tmp))]
        offspring = offspring +  [copy.deepcopy(best_populations[t]) for t in range(len(best_populations))]
    
#     print("****************after shuffle*******************")
    fitness_list_tmp = [cal_fitness(offspring[j], breaks_offspring[j], N) for j in range(POPULATION)]
#     print('in IPGA Cal',min(fitness_list_tmp))
#     print("-------------------IPGA END----------------------")
    return offspring, breaks_offspring

In [561]:
iters = 10000
def ga_start(max_iters):
    min_nodes = 2
    max_nodes = 40
#     max_nodesodes = None 
    populations, breaks = generate_populations(POPULATION, N, M, min_nodes, max_nodes)
    global_best = sys.maxsize
    
    fitness_val = []
    print("min:{}, max:{}".format(min_nodes, max_nodes))
    for i in range(max_iters):
        fitness_list = [cal_fitness(populations[j], breaks[j], N) for j in range(POPULATION)]

        global_best = min(global_best,min(fitness_list))
        
        print(global_best)
        populations, breaks= IPGA(populations, breaks, fitness_list, N, M, min_nodes, max_nodes)

#     print(global_best)
        
for i in range(1):
    ga_start(iters)




min:2, max:40
1509.5968719385792
1413.002210236659
1413.002210236659
1383.1794105221286
1347.1745658973211
1343.6592191079776
1316.2881692873593
1297.9667569406083
1297.9667569406083
1297.9667569406083
1279.8124215322173
1276.4312126946752
1276.4312126946752
1276.4312126946752
1248.076955714666
1247.5058271713065
1247.5058271713065
1217.281190651375
1217.281190651375
1217.281190651375
1217.281190651375
1188.7722385890984
1164.1050107797496
1149.0304074761914
1149.0304074761914
1145.5018042010804
1145.5018042010804
1132.9066547303892
1132.9066547303892
1132.9066547303892
1112.6397208443743
1112.6397208443743
1112.6397208443743
1112.6397208443743
1112.6397208443743
1112.6397208443743
1104.4919760118546
1104.4919760118546
1104.4919760118546
1104.4919760118546
1099.700733024626
1099.700733024626
1099.700733024626
1099.700733024626
1099.700733024626
1092.6138994145078
1092.6138994145078
1079.8170456273092
1079.8170456273092
1066.4976091860763
1066.4976091860763
1056.623619053753
1046.575439

764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.8689085372655
764.868908

703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.7813720017367
703.781372

690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
690.0618697411684
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.0248423328543
670.024842

661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
661.6448050934242
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.4957892731105
658.495789

...

581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.2287330023079
581.228733

577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.169408

577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.1694089420491
577.169408

575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.120126

575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.1201260568869
575.120126

In [487]:
b = [11, 13, 28, 42]
p = [42, 15, 31, 6, 3, 44, 23, 37, 27, 8, 19, 24, 25, 0, 47, 46, 22, 39, 33, 34, 35, 1, 20, 29, 12, 50, 38, 7, 48, 45, 26, 41, 2, 30, 11, 21, 4, 13, 17, 36, 10, 5, 18, 49, 14, 40, 28, 43, 16, 32, 9]
cal_fitness(p, b, N)

1650.4482546973682